In [6]:
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import time
import xarray as xr

#### Filepaths and Filelist

In [7]:
#ars path
# path = '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent_data_downsize/nc/2013/*.nc'
#mm path
path = '/Users/megmason/Desktop/nc_2014/*.nc'
flist = glob.glob(path)
print(type(flist))
print(flist)

<class 'list'>
['/Users/megmason/Desktop/nc_2014/20140323_SUPERsnow_depth_3m.nc', '/Users/megmason/Desktop/nc_2014/20140407_SUPERsnow_depth_3m.nc', '/Users/megmason/Desktop/nc_2014/20140413_SUPERsnow_depth_3m.nc', '/Users/megmason/Desktop/nc_2014/20140420_SUPERsnow_depth_3m.nc', '/Users/megmason/Desktop/nc_2014/20140428_SUPERsnow_depth_3m.nc', '/Users/megmason/Desktop/nc_2014/20140502_SUPERsnow_depth_3m.nc', '/Users/megmason/Desktop/nc_2014/20140511_SUPERsnow_depth_3m.nc', '/Users/megmason/Desktop/nc_2014/20140517_SUPERsnow_depth_3m.nc', '/Users/megmason/Desktop/nc_2014/20140527_SUPERsnow_depth_3m.nc', '/Users/megmason/Desktop/nc_2014/20140531_SUPERsnow_depth_3m.nc', '/Users/megmason/Desktop/nc_2014/20140605_SUPERsnow_depth_3m.nc']


#### Create and Close XarrayDataset

In [8]:
ds=xr.open_mfdataset(flist, concat_dim='time') #dataset
ds.close()

#### float to int (still testing...IF RUN, LOOSES 'BAND1' (??)

In [9]:
ds=ds['Band1'].astype(np.int16, copy=False)
ds

<xarray.DataArray 'Band1' (time: 11, y: 16781, x: 17568)>
dask.array<shape=(11, 16781, 17568), dtype=int16, chunksize=(1, 16781, 17568)>
Coordinates:
  * x        (x) float64 2.54e+05 2.54e+05 2.54e+05 ... 3.067e+05 3.067e+05
  * y        (y) float64 4.179e+06 4.179e+06 4.179e+06 ... 4.23e+06 4.23e+06
Dimensions without coordinates: time

#### Specs. on Dataset (ds)

In [ ]:
# few others....but these are main
ds.dims
ds.values
ds.coords
# ds.Band1.shape

#### Assign time coordinate baised on lidar flight date & select time by year (or specific date)

In [10]:
date=[]

for f in sorted(flist):
#     print(f)
    dt_str = f.split("/")[-1] #splits on / and saves the last one
    dt_str = "".join([c for c in dt_str if c.isnumeric()]) #grabs numeric values
    dt = pd.to_datetime(dt_str[:8]) #convert to datetime, grabs digits for dates only
#     print('file:',dt.date())
    date.append(dt.date())

time_var = pd.to_datetime(date)
ds['DATE'] = (['time'], time_var) #explain this...
ds = ds.assign_coords(time=ds.DATE) #assigns coord, gives DATE (datetime) to each layer in stack

ds


<xarray.DataArray 'Band1' (time: 11, y: 16781, x: 17568)>
dask.array<shape=(11, 16781, 17568), dtype=int16, chunksize=(1, 16781, 17568)>
Coordinates:
  * x        (x) float64 2.54e+05 2.54e+05 2.54e+05 ... 3.067e+05 3.067e+05
  * y        (y) float64 4.179e+06 4.179e+06 4.179e+06 ... 4.23e+06 4.23e+06
    DATE     (time) datetime64[ns] 2014-03-23 2014-04-07 ... 2014-06-05
  * time     (time) datetime64[ns] 2014-03-23 2014-04-07 ... 2014-06-05

In [11]:
#SELECT YEAR, or DATE
# ds = ds.sel(time='2015')
ds = ds.sel(time='2014')

In [12]:
dc = ds['DATE'].chunks
d = ds.chunks

print(d)

((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), (16781,), (17568,))


In [ ]:
d=ds['Band1'].chunks #dataArray
d

#### compute stuff (testing)

In [13]:
# variance in snow depth
v = ds.Band1.var(dim='time') #compute variance on every pixel in the time dimention 
v = np.where(v>1, v, np.nan) # filters everything greater than 1cm (i.e gives alpha bg)
print(v)










AttributeError: 'DataArray' object has no attribute 'Band1'

In [ ]:
# t=time.time()

# variance in snow depth
varsn = ds.Band1.var(dim='time') #compute variance on every pixel in the time dimention 
varsn = np.where(varsn>.001, varsn, np.nan) # filters everything greater than 1mm (i.e gives alpha bg)
print(varsn)
# fig = plt.figure(figsize=(15, 10))
# plt.imshow(varsn, cmap='jet', origin='lower', vmin=.05, vmax=1.0) #[::-1,:] - reverses it
# plt.title('snow depth [3m] variance, 2018')
# plt.colorbar()
# plt.savefig('figs/var_13-16', dpi=600)

# tt=time.time()
# print('runtime =', (tt-t)/60)